## Configuring SRS
In this example we demonstrate the configuration SRS (structured result data) you can upsert any valid content, but the SRS is designed to store external result data that can feed into LUSID valuation operations. 

## Imports

In [9]:

import io
import os
import pandas as pd
pd.options.mode.chained_assignment = None
import json
import pytz
from IPython.core.display import HTML
from datetime import datetime

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid
import lusid as lu
import lusid.api as la
import lusid.models as lm
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.12546.0,0.5.3481,"{'relation': 'RequestLogs', 'href': 'https://f..."


# Setup

We must first set up some parameters and initialize the LUSID APIs. Then we will upload the instruments master, create the properties and portfolios and finally upload the holdings.

In [10]:
scope = "srs-demo-feb-2024"
test_bond_client_internal1 = "XS1698906259"
test_bond_client_internal2 = "XS2062820290"
test_bond_client_internal3 = "XS2163089563"

portfolio_code = "SRS-Instrument-Test"

holdings_api = api_factory.build(lu.TransactionPortfoliosApi)
srs_api = api_factory.build(lu.StructuredResultDataApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
aggregation_api = api_factory.build(lu.AggregationApi)
configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)
property_definitions_api = api_factory.build(lu.PropertyDefinitionsApi)

# Create instrument

In [11]:
"""
-----------
Create Bond
-----------

PREREQUISITE: Run the code in the 'Creating an API Factory' snippet to create an API factory.

In this snippet we create a Bond using the UpsertInstruments endpoint:

https://www.lusid.com/docs/api/#operation/UpsertInstruments

"""

bond_definition1 = lm.Bond(
    start_date="2023-11-01",
    maturity_date="2028-06-30",
    dom_ccy="GBP",
    flow_conventions=lm.FlowConventions(
        currency="GBP",
        payment_frequency="6M",
        day_count_convention="Act365",
        roll_convention="None",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=2,
        reset_days=2,
    ),
    principal=1,
    coupon_rate=0.04,
    identifiers={},
    instrument_type="Bond",
)

bond_definition2 = lm.Bond(
    start_date="2023-11-01",
    maturity_date="2035-10-10",
    dom_ccy="GBP",
    flow_conventions=lm.FlowConventions(
        currency="GBP",
        payment_frequency="6M",
        day_count_convention="Act365",
        roll_convention="ModifiedFollowing",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=2,
        reset_days=2,
    ),
    principal=1,
    coupon_rate=0.02,
    identifiers={},
    instrument_type="Bond",
)

bond_definition3 = lm.Bond(
    start_date="2023-11-01",
    maturity_date="2030-04-27",
    dom_ccy="GBP",
    flow_conventions=lm.FlowConventions(
        currency="GBP",
        payment_frequency="6M",
        day_count_convention="Act365",
        roll_convention="None",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=2,
        reset_days=2,
    ),
    principal=1,
    coupon_rate=0.03,
    identifiers={},
    instrument_type="Bond",
)


upsert_instrument = instruments_api.upsert_instruments(
    request_body={
        "ABDNLN 4 1/4 06/30/28": lm.InstrumentDefinition(
            name="ABDNLN 4 1/4 06/30/28",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="XS1698906259")},
            definition=bond_definition1),
            "CDTFIN 2 1/4 10/10/35": lm.InstrumentDefinition(
            name="CDTFIN 2 1/4 10/10/35",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="XS2062820290")},
            definition=bond_definition2),
         "TSCOLN 2 3/4 04/27/30": lm.InstrumentDefinition(
            name="TSCOLN 2 3/4 04/27/30",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="XS2163089563")},
            definition=bond_definition3),
    }
)


upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instrument.values.values()))
display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])

,name,lusid_instrument_id
0,ABDNLN 4 1/4 06/30/28,LUID_00003D9Y
1,TSCOLN 2 3/4 04/27/30,LUID_00003D9W
2,CDTFIN 2 1/4 10/10/35,LUID_00003D9X


## Create portfolio

In [12]:
"""
-------------------------------
Create a Transaction Portfolio:
-------------------------------

PREREQUISITE: Run the code in the 'Creating an API Factory' snippet to create an API factory.

In this snippet we create a Transaction Portfolio using the CreatePortfolio endpoint:

https://www.lusid.com/docs/api/#operation/CreatePortfolio

"""


# Create a Transactions Portfolios API
transaction_portfolio_api = api_factory.build(la.TransactionPortfoliosApi)

# Attempt to create a Transaction Portfolio
try:
    create_portfolio_response = transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="GBP",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

    print(create_portfolio_response)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])


Could not create a portfolio with id 'SRS-Instrument-Test' because it already exists in scope 'srs-demo-feb-2024'.


# Set holding

In [13]:
transaction_portfolio_api = api_factory.build(la.TransactionPortfoliosApi)

try:
    set_holdings = transaction_portfolio_api.set_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at="2024-01-01",
        adjust_holding_request=[
            lm.AdjustHoldingRequest(
                instrument_identifiers={"Instrument/default/ClientInternal": test_bond_client_internal1},
                sub_holding_keys=None,
                properties=None,
                tax_lots=[lm.TargetTaxLotRequest(units=10000)],
                currency="GBP",
            ),
             lm.AdjustHoldingRequest(
                instrument_identifiers={"Instrument/default/ClientInternal": test_bond_client_internal2},
                sub_holding_keys=None,
                properties=None,
                tax_lots=[lm.TargetTaxLotRequest(units=5000)],
                currency="GBP",
            ),
             lm.AdjustHoldingRequest(
                instrument_identifiers={"Instrument/default/ClientInternal": test_bond_client_internal3},
                sub_holding_keys=None,
                properties=None,
                tax_lots=[lm.TargetTaxLotRequest(units=3000)],
                currency="GBP",
            )
        ]
    )

    print(set_holdings)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])


{'href': 'https://fbn-oscar.lusid.com/api/api/transactionportfolios/srs-demo-feb-2024/SRS-Instrument-Test/holdings?asAt=2024-02-05T14%3A42%3A32.6973310%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://fbn-oscar.lusid.com/api/api/portfolios/srs-demo-feb-2024/SRS-Instrument-Test?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2024-02-05T14%3A42%3A32.6973310%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://fbn-oscar.lusid.com/api/api/transactionportfolios/srs-demo-feb-2024/SRS-Instrument-Test/holdings?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2024-02-05T14%3A42%3A32.6973310%2B00%3A00',
            'method': 'GET',
            'relation': 'Holdings'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-oscar.lusid.com/app/insights/logs

# Data map

Now that we have our dataset with our in-house valuations (the MV and GainLoss columns above), we can map these and upload them to Lusid.

In [14]:
srs_data_map = lm.DataMapping(
    data_definitions=[
        # Define key to identify a single result
        ## This address key can be what ever we want
        lm.DataDefinition(address="UnitResult/LusidInstrumentId", data_type="string", name="LUID", key_type="Unique"),
        
        lm.DataDefinition(address="UnitResult/Spread", name="Spread", data_type="decimal", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/Duration", name="Duration", data_type="decimal", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/YTM", name="YTM", data_type="decimal", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/DTM", name="DTM", data_type="decimal", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/AccruedFactor", name="AccruedFactor", data_type="decimal", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/AccruedDays", name="AccruedDays", data_type="decimal", key_type="Leaf"),
        
        lm.DataDefinition(address="UnitResult/Valuation/PV", data_type="Result0D", key_type="CompositeLeaf"),
        lm.DataDefinition(address="UnitResult/Valuation/PV/Amnt", data_type="decimal", key_type="Leaf", name="NAV"),
        lm.DataDefinition(address="UnitResult/Valuation/PV/Ccy", data_type="string", key_type="Leaf", name="NAVCcy"),
        lm.DataDefinition(address="UnitResult/Valuation/Exposure", data_type="Result0D", key_type="CompositeLeaf"),
        lm.DataDefinition(address="UnitResult/Valuation/Exposure/Amount", data_type="decimal", key_type="Leaf", name="ExposureAmount"),
        lm.DataDefinition(address="UnitResult/Valuation/Exposure/Ccy", data_type="string", key_type="Leaf", name="ExposureCcy"),
        
    ]
)

# Because the data maps are immutable, we must increase the version number each time we make any changes and upload a new version
srs_data_map_key = lm.DataMapKey(version="0.0.13", code="srs-test-2024-01-29")

# Once the map has been completed   , we will create the data map in LUSID
try:
    response = srs_api.create_data_map(
        scope=scope,
        request_body={
            "market-valuation-map": lm.CreateDataMapRequest(
                id=srs_data_map_key,
                data=srs_data_map
            )
        }
    )
    display(response)    
except lu.ApiException as e:
    if json.loads(e.body)["code"] == 461:
        display("DataMaps are immutable - a datamap under this key already exists")
    else:
        display(json.loads(e.body))

'DataMaps are immutable - a datamap under this key already exists'

## Upsert Data
adding market data values for 2 different dates

In [15]:
data = {
    #bond 1, bond 2, bond 3
    "LUID": upsert_instruments_response_df['lusid_instrument_id'],
    "Duration": [4, 9.91, 5.7],
    "Spread": [1.2, 2.3, 1.9],
    "YTM": [0.6, 0.7, 0.8],
    "DTM": [5.7, 10.1, 6.7],
    "AccruedFactor": [1.343, 1.43, 1.3],
    "AccruedDays" : [25, 24, 15],
    "NAV" : [123456, 654321, 13540],
    "NAVCcy" : ['GBP', 'GBP', 'GBP'],
    "ExposureAmount" : [654321, 123456, 135400],
    "ExposureCcy" : ['GBP','GBP', 'GBP']
}

srs_df = pd.DataFrame(data)

srs_df

,LUID,Duration,Spread,YTM,DTM,AccruedFactor,AccruedDays,NAV,NAVCcy,ExposureAmount,ExposureCcy
0,LUID_00003D9Y,4.00,1.20,0.60,5.70,1.34,25,123456,GBP,654321,GBP
1,LUID_00003D9W,9.91,2.30,0.70,10.10,1.43,24,654321,GBP,123456,GBP
2,LUID_00003D9X,5.70,1.90,0.80,6.70,1.30,15,13540,GBP,135400,GBP


In [16]:
effective_at = "2024-01-01"
document_code = "MarketValuation001"
result_type = "UnitResult/Analytic"
source = "Client"

srs_data_id = lm.StructuredResultDataId(
    source = source,
    code = document_code,
    effective_at = effective_at,
    result_type = result_type
)


srs_data = lm.StructuredResultData(
    document_format="Csv",
    version="0.0.1",
    name="Market valuations",
    data_map_key=srs_data_map_key,
    document=srs_df.to_csv(index=False)      
)

srs_api.upsert_structured_result_data(
    scope=scope, 
    request_body={ 
        "data": lm.UpsertStructuredResultDataRequest(
            id=srs_data_id, 
            data=srs_data
        )
    }
)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-oscar.lusid.com/app/insights/logs/0HN16H2J3C1CP:00000257',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'data': datetime.datetime(2024, 2, 6, 9, 23, 46, 65098, tzinfo=tzlocal())}}

In [17]:
data2 = {
    #bond 1, bond 2, bond 3
    "LUID": upsert_instruments_response_df['lusid_instrument_id'],
    "Duration": [3, 9.8, 5.7],
    "Spread": [1.2, 2.3, 1.9],
    "YTM": [0.6, 0.7, 0.8],
    "DTM": [5.7, 10.1, 6.7],
    "AccruedFactor": [1.323, 1.43, 1.3],
    "AccruedDays" : [25, 24, 15],
    "NAV" : [123356, 654121, 13140],
    "NAVCcy" : ['GBP', 'GBP', 'GBP'],
    "ExposureAmount" : [653321, 122456, 134400],
    "ExposureCcy" : ['GBP','GBP', 'GBP']
}

srs_df2 = pd.DataFrame(data2)

srs_df2

,LUID,Duration,Spread,YTM,DTM,AccruedFactor,AccruedDays,NAV,NAVCcy,ExposureAmount,ExposureCcy
0,LUID_00003D9Y,3.00,1.20,0.60,5.70,1.32,25,123356,GBP,653321,GBP
1,LUID_00003D9W,9.80,2.30,0.70,10.10,1.43,24,654121,GBP,122456,GBP
2,LUID_00003D9X,5.70,1.90,0.80,6.70,1.30,15,13140,GBP,134400,GBP


In [18]:
effective_at = "2024-01-02"
document_code = "MarketValuation001"
result_type = "UnitResult/Analytic"
source = "Client"

srs_data_id = lm.StructuredResultDataId(
    source = source,
    code = document_code,
    effective_at = effective_at,
    result_type = result_type
)


srs_data = lm.StructuredResultData(
    document_format="Csv",
    version="0.0.1",
    name="Market valuations",
    data_map_key=srs_data_map_key,
    document=srs_df.to_csv(index=False)      
)

srs_api.upsert_structured_result_data(
    scope=scope, 
    request_body={ 
        "data": lm.UpsertStructuredResultDataRequest(
            id=srs_data_id, 
            data=srs_data
        )
    }
)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-oscar.lusid.com/app/insights/logs/0HN16G2J880M7:00000007',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'data': datetime.datetime(2024, 2, 6, 9, 23, 49, 336277, tzinfo=tzlocal())}}

## Read from SRS

We check if everything has been uploaded correctly by retrieving the previously uploaded data.
Here we can see that our upload was successful and the structured results store contains our data.

The Structured Result Store allows us to upsert structured results, but to give them structure we must first define a Data Mapping,

You can think of a Data Mapping as defining the columns and the column types for each row that gets upserted into the SRS using a specific Data Map

Each "row" can have multiple keys and when difining the data map there are 4 different key types, Unique, PartOfUnique, Leaf, and CompositeLeaf


Unique
A primary key that will throw a unique key constraint if multiple equal values are upserted for a single specific result type


Part of Unique
A key that will be considered as part of the compoisite primary key for an entity


Leaf
Leaf define a value that wont be considered as part of the primary or primary composite key.


Composite Leaf
CompositeLeaf is an abstraction that allows the user to specify which Leafs should be connected, they define data types rather than names of entities as they represent a group of entities. E.g. If we have an Accural we can define a composite leaf as two leafs of amount and currency


"UnitResult/Accrual"
DataDefinition(address="UnitResult/Accrual", dataType= "Result0D", keyType="CompositeLeaf")

"UnitResult/Accrual/Amount"
DataDefinition(address="UnitResult/Accrual/Amount", name="Accrual", dataType= "decimal", keyType="Leaf")

"UnitResult/Accrual/Ccy"
DataDefinition(address="UnitResult/Accrual/AmountCcy", name="AccrualCcy", dataType= "string", keyType="Leaf")

In [19]:
srs_data_id

{'code': 'MarketValuation001',
 'effective_at': '2024-01-02',
 'result_type': 'UnitResult/Analytic',
 'source': 'Client'}

In [20]:
scope

'srs-demo-feb-2024'

In [40]:
#datamap that defines a key for each column
display(srs_api.get_data_map(scope=scope, request_body={"id1": srs_data_map_key}).values)


{'id1': {'data_definitions': [{'address': 'UnitResult/LusidInstrumentId',
                        'allow_missing': False,
                        'allow_null': False,
                        'data_type': 'Unique',
                        'key_type': 'String',
                        'name': 'LUID'},
                       {'address': 'UnitResult/Spread',
                        'allow_missing': False,
                        'allow_null': False,
                        'data_type': 'Leaf',
                        'key_type': 'Decimal',
                        'name': 'Spread'},
                       {'address': 'UnitResult/Duration',
                        'allow_missing': False,
                        'allow_null': False,
                        'data_type': 'Leaf',
                        'key_type': 'Decimal',
                        'name': 'Duration'},
                       {'address': 'UnitResult/YTM',
                        'allow_missing': False,
                        'a

In [22]:
#display SRS data values
key = f"{srs_data_id.code}-{effective_at}"

values = srs_api.get_structured_result_data(
    scope=scope, 
    request_body={
        key: srs_data_id
    }
)

s = io.StringIO(values.values[key].document)

values_df = pd.read_csv(s)
display(values_df)

,LUID,Duration,Spread,YTM,DTM,AccruedFactor,AccruedDays,NAV,NAVCcy,ExposureAmount,ExposureCcy
0,LUID_00003D9Y,4.00,1.20,0.60,5.70,1.34,25,123456,GBP,654321,GBP
1,LUID_00003D9W,9.91,2.30,0.70,10.10,1.43,24,654321,GBP,123456,GBP
2,LUID_00003D9X,5.70,1.90,0.80,6.70,1.30,15,13540,GBP,135400,GBP


In [23]:
%%luminesce

@lookup_table = 
select 'srs-example' as Scope, 
'MarketValuation001' as Code, 
'Client' as Source, 
'UnitResult/Analytic' as ResultType, 
#2024-01-01# as EffectiveAt;
select * from Lusid.UnitResult.StructuredResult where toLookUp = @lookup_table;

,Scope,Code,Name,Source,ResultType,EffectiveAt,Version,Document,Format,DataMapCode,DataMapVersion,Error
0,srs-example,MarketValuation001,NaN,Client,UnitResult/Analytic,2024-01-01,NaN,NaN,Unknown,NaN,NaN,NaN


# Valuation Engine
Now we will call the GetValuation API to perform a valuation where i can request metrics from the SRS as well or instead of built-in LUSID. 

In [24]:

recipe_code = "MarketValuation"

recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    pricing=lm.PricingContext(
        result_data_rules=[
            lm.ResultDataKeyRule(
                resource_key="UnitResult/*",
                supplier=source,
                data_scope=scope,
                document_code=document_code,
                quote_interval="5D",
                document_result_type=result_type,
                result_key_rule_type="ResultDataKeyRule"
            )
        ],
        options = lm.PricingOptions(
            allow_partially_successful_evaluation=True,
            allow_any_instruments_with_sec_uid_to_price_off_lookup=False
        )                              
    )
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = lm.UpsertRecipeRequest(
        configuration_recipe = recipe
    )
)

In [25]:
def run_valuation(portfolio_code, effective_at):
    
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=effective_at.isoformat()),
        metrics=[
            lm.AggregateSpec("Instrument/default/LusidInstrumentId", "Value"),
            lm.AggregateSpec("UnitResult/ModifiedDuration", "Value"),
            lm.AggregateSpec("UnitResult/Spread", "Value"),
            lm.AggregateSpec("UnitResult/YTM", "Value"),
            # lm.AggregateSpec("UnitResult/Analytic/ModifiedDuration", "Value"),
            # lm.AggregateSpec("Analytic/ModifiedDuration", "Value")
        ]
    )
    
    return aggregation_api.get_valuation(valuation_request=valuation_request)

In [26]:
recipe_code

'MarketValuation'

In [28]:
day1 = datetime(2024, 1, 1, tzinfo=pytz.utc)

valuation=run_valuation(portfolio_code, day1)
pd.DataFrame(valuation.data)

,Instrument/default/LusidInstrumentId,UnitResult/ModifiedDuration,UnitResult/Spread,UnitResult/YTM,Aggregation/Errors
0,LUID_00003D9Y,None,1.20,0.60,[UnitResult/ModifiedDuration:UnitResult/Portfo...
1,LUID_00003D9X,None,1.90,0.80,[UnitResult/ModifiedDuration:UnitResult/Portfo...
2,LUID_00003D9W,None,2.30,0.70,[UnitResult/ModifiedDuration:UnitResult/Portfo...
